## Setup
- load train and test datasets, create TensorFlow Datasets from them
- enable eager execution for TF for easier prototyping
- define main architecutre params

In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
import skimage

from src.data_utils import Dataset
from src.network import unet_3d

tf.enable_eager_execution()

In [25]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from src.data_utils import Dataset
from src.network import unet_3d

tf.enable_eager_execution()

train_dataset = Dataset.load_dataset('../data/processed/train_dataset.pckl')
test_dataset = Dataset.load_dataset('../data/processed/test_dataset.pckl')
x_train = [p.scans for p in train_dataset.patients.values()]
y_train = [p.seg for p in train_dataset.patients.values()]
x_test = [p.scans for p in test_dataset.patients.values()]
y_test = [p.seg for p in test_dataset.patients.values()]

i = 28
inputs = x_train[i]
outputs = y_train[i]

# turn them into tensors
inputs = tf.constant(inputs.reshape((1, inputs.shape[0], inputs.shape[1], inputs.shape[2], 1)))
outputs = tf.constant(outputs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
y_pred = unet_3d(inputs)


conv1: (1, 24, 128, 128, 16)
conv2: (1, 24, 128, 128, 32)
maxpool layer: (1, 12, 64, 64, 32)
conv1: (1, 12, 64, 64, 32)
conv2: (1, 12, 64, 64, 64)
maxpool layer: (1, 6, 32, 32, 64)
conv1: (1, 6, 32, 32, 64)
conv2: (1, 6, 32, 32, 128)
maxpool layer: (1, 3, 16, 16, 128)
conv1: (1, 3, 16, 16, 128)
conv2: (1, 3, 16, 16, 256)
upconv layer: (1, 6, 32, 32, 256)
concat layer: (1, 6, 32, 32, 128)
up_conv layer1: (1, 6, 32, 32, 128)
up_conv layer2: (1, 6, 32, 32, 128)
upconv layer: (1, 12, 64, 64, 128)
concat layer: (1, 12, 64, 64, 64)
up_conv layer1: (1, 12, 64, 64, 64)
up_conv layer2: (1, 12, 64, 64, 64)
upconv layer: (1, 24, 128, 128, 64)
concat layer: (1, 24, 128, 128, 32)
up_conv layer1: (1, 24, 128, 128, 32)
up_conv layer2: (1, 24, 128, 128, 32)
output layer: (1, 24, 128, 128, 3)


In [27]:
class_weights = tf.constant([[1.0, 200.0, 200.0]])
w = tf.gather(class_weights, outputs)
tf.losses.softmax_cross_entropy(outputs, y_pred.numpy()[0,:,:,:,:])

<tf.Tensor: id=5217, shape=(), dtype=float32, numpy=1.2470297>

In [157]:
import  tensorflow as tf
import numpy as np

np.random.seed(123)
sess = tf.InteractiveSession()

# let's say we have the logits and labels of a batch of size 6 with 5 classes
logits = tf.constant(np.random.randint(0, 10, 30).reshape(6, 5), dtype=tf.float32)
labels = tf.constant(np.random.randint(0, 5, 6), dtype=tf.int32)

# specify some class weightings
class_weights = tf.constant([0.3, 0.1, 0.2, 0.3, 0.1])

# specify the weights for each sample in the batch (without having to compute the onehot label matrix)
weights = tf.gather(class_weights, labels)

# compute the loss
tf.losses.sparse_softmax_cross_entropy(labels, logits, weights)

/home/daniel/.virtualenvs/tf/lib/python3.5/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


<tf.Tensor: id=5112, shape=(), dtype=float32, numpy=0.77875894>